In [1]:
### https://tom-toolkit.readthedocs.io/en/stable/advanced/scripts.html

In [2]:
from tom_targets.models import Target
t = Target.objects.create(name='m51', type='SIDEREAL', ra=123.3, dec=23.3)
print(t)

Target post save hook: m51 created: True


m51


In [3]:
t.save(extras={'foo': 42, 'bar': 'baz'}, names=['Messier 51'])
print(t.extra_fields)

Target post save hook: m51 created: False


{}


In [4]:
target = Target.objects.get(name='m51')
print(target)

m51


In [5]:
target.ra

123.3

In [6]:
target.future_observations

[]

In [7]:
target.names

['m51', 'Messier 51']

In [8]:
target.delete()

(4,
 {'tom_targets.TargetName': 1,
  'tom_targets.TargetExtra': 2,
  'tom_targets.TargetList_targets': 0,
  'tom_dataproducts.ReducedDatum': 0,
  'tom_targets.Target': 1})

In [9]:
from tom_targets.models import Target
from tom_observations.facilities.lco import LCOFacility, LCOBaseObservationForm

In [10]:
target = Target.objects.get(name='Snickerdoodle Star')
print(target.ra,target.dec)
print(target.id,target.future_observations,target.names)

12.7358486 5.663694444444444
3 [] ['Snickerdoodle Star']


In [15]:
form = LCOBaseObservationForm({
    'name': 'Programmatic Observation',
    'proposal': 'DDT2019B-006',
    'ipp_value': 1.05,
    'start': '2019-10-09T00:00:00',
    'end': '2019-10-10T00:00:00',
    'filter': 'R',
    'instrument_type': '1M0-SCICAM-SINISTRO',
    'exposure_count': 1,
    'exposure_time': 20,
    'max_airmass': 4.0,
    'observation_mode': 'NORMAL',
    'target_id': target.id,
    'facility': 'LCO'
})

In [16]:
form.is_valid()

True

In [17]:
observation_ids = LCOFacility().submit_observation(form.observation_payload())
print(observation_ids)

[1944967]


In [18]:
from tom_observations.models import ObservationRecord

In [19]:
for observation_id in observation_ids:
    record = ObservationRecord.objects.create(
        target=target,
        facility='LCO',
        parameters=form.serialize_parameters(),
        observation_id=observation_id
    )
    print(record)

Observation change state hook: Snickerdoodle Star @ LCO from None to 


Snickerdoodle Star @ LCO


In [20]:
print(target.id,target.future_observations,target.names)

3 [<ObservationRecord: Snickerdoodle Star @ LCO>] ['Snickerdoodle Star']
